In [ ]:
from pathlib import Path
import pandas as pd
import numpy  as np

# Leggiamo i dati di APPA

Caricamento dei file csv in una tabella chiamata "appa".

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
data_folder = Path("/content/drive/MyDrive/data")

In [ ]:
appa = pd.read_csv(
    data_folder / "DataSet_APPA/processed/appa_data.csv", parse_dates=["Data"]
)
appa

## Controlliamo le statistiche dei dati

Controlliamo per ogni stazione la data della prima rilevazione, di fine e il numero di rilevazioni.

In [ ]:
appa.groupby("Stazione").Data.agg(["min", "max", "count"])
# appa.groupby("Inquinante").Data.agg(["min", "max", "count"])

Raggruppiamo anche per inquinante in modo da poter osservare gli intervalli di rilevazione dei singoli inquinanti.

In [ ]:
appa.groupby("Inquinante").Data.agg(["min", "max", "count"])

Raggruppiamo per inquinante e per stazione.

In [ ]:
appa.groupby(["Stazione", "Inquinante"]).Data.agg(["min", "max", "count"])

# Selezioniamo una singola stazione

Visualizziamo tutti i dati presenti nella tabella.

In [ ]:
appa

Scegliamo una stazione, per esempio la Piana Rotaliana e grafichiamo la concentrazione di inquinante in funzione del tempo. Potete provare a cambiare stazione.



```
station = "Piana Rotaliana"


for g_id, g in appa[appa.Stazione == station].groupby(["Inquinante"]):
    g.plot.scatter(
        x="Data",
        y="Valore",
        figsize=(10, 3),
        title=g_id[0],
        alpha=0.05,
        marker=".",
    )
```



## Visualizziamo i dati giornalieri, settimanali, mensili, annuali

`appa.Stazione == station` serve per filtrare

`resample("W")` serve per raggruppare settimanalmente

```python

for g_id, g in appa[appa.Stazione == station].groupby(["Inquinante"]):
    g.set_index("Data").resample("W").mean("Valore").reset_index().plot(
        x="Data",
        y="Valore",
        figsize=(12, 3),
        title=g_id[0],
        alpha=0.5,
        marker=".",
    )
```

Provate ore a raggruppare i dati giornalmente (D), mensilmente (M), annualmente (Y).

# Selezioniamo un singolo inquinante
Provate a visualizzare i dati per uno stesso inquinante in stazioni diverse modificando il codice precedente.



```
station = nome stazione

for g_id, g in appa[appa.Stazione == station].groupby(["Inquinante"]):
    g.plot.scatter(
        x="Data",
        y="Valore",
        figsize=(10, 3),
        title=g_id[0],
        alpha=0.05,
        marker=".",
    )
```



Ora possiamo provare nei grafici precenti a zoomare su un dato intervallo temporale e ad aggiungere una linea orizzontale che rappresenta il valore soglia di PM10



```
pol = "PM10"

for g_id, g in appa[appa.Inquinante == pol].groupby("Stazione"):
    g = (
        g.sort_values("Data")
        .set_index("Data")
        .loc["2023-01-01":"2023-12-31"]
        .reset_index()
    )
    
    ax = g.plot.scatter(
        x="Data",
        y="Valore",
        figsize=(10, 3),
        title=g_id,
        alpha=0.05,
        marker=".",
        # # use global maximum to compare stations
        # ylim=(-5, appa.loc[appa.Inquinante == pol, "Valore"].max())
    )

    ax.axhline(50, c="r", linestyle=":")

```








## Visualizziamo i dati giornalieri, settimanali, mensili e annuali mettendo in uno stesso grafico min, max e media



```
pol = "PM10"

for g_id, g in appa[appa.Inquinante == pol].groupby("Stazione"):
    g = (
        g.sort_values("Data")
        .set_index("Data")
        .resample("M")
        .Valore
        .agg(["mean", "max", "min"])
    )

    ax = g.plot.line(
        figsize=(10, 3),
        title=g_id,
    )
```



# Piana Rotaliana e PM10

Decidiamo di lavorare sulla stazione Piana Rotaliana per riuscire a prevedere il PM10.

Andiamo a vedere quante volte viene superata una determinata soglia e calcoliamo la percentuale di superamento della soglia.

```
soglia = 100

rotaliana = appa[
    (appa.Stazione == "Piana Rotaliana")
    & (appa.Inquinante == "PM10")
]


superamento = appa[
    (appa.Stazione == "Piana Rotaliana")
    & (appa.Inquinante == "PM10")
    & (appa.Valore > soglia)
]
```

Per vedere la lunghezza di una tabella si utilizza il comando


```
print(len(nome_tabella))

```


In Italia il parametro della valutazione di PM10 è la **media giornaliera**: secondo il D. Lgs. 155/2010 questo limite è pari a 50 µg/mc, da non superare più di **35 volte per anno civile**. Nello stesso decreto viene anche stabilito un limite annuale fissato a 40 µg/mc come media annua.

Calcoliamo quante volte viene superata la media giornaliera.

Copia il seguente codice per ricavare le medie giornalierie dalla tabella "appa" e poi calcola come prima quante volte viene superato il valore.

```
exp_data = appa.loc[(appa.Stazione == station)].set_index("Data").groupby(["Inquinante"]).resample("D").mean("Valore").reset_index()
```





Utilizziamo l'operatore logico & per selezionare anno, inquinante e soglia come nell'esempio riportato di seguito.


```
exp_data_anno = exp_data.loc[(exp_data.Data.dt.year == 2015) & (exp_data.Inquinante == "PM10") & (exp_data.Valore > soglia)]
```

## Correlazione fra gli inquinanti

Conoscere la correlazione fra gli inquinanti potrebbe essere utile al decisore politico. Per calcolare la correlazione dobbiamo trasformare la tabella "appa", in una tabella con una colonna per ogni tipo di inquinante.

Utilizziamo il seguente codice per manipolare la tabella.

```
# converto i nomi degli inquinanti in codici (formule chimiche o abbreviazioni)
exp_data["Inquinante"] = exp_data.Inquinante.map(
    {"PM10": "PM10", "Biossido di Azoto": "NO2", "Ozono": "O3"}
)

# converto il dataframe in una tabella pivot (formato wide) e ordino i dati per data
exp_data = exp_data.pivot_table(
    index="Data", columns="Inquinante", values="Valore"
).sort_index()

exp_data
```



Per calcolare la tabella di correlazione si utilizza il seguente comando:

```
exp_data.corr()
```



# Osservazioni

Riporta le tue eventuali osservazioni

# Possibili domande e spunti

- Si può predire la concentrazione futura data quella passata?

- Quali sono le variabili che hanno un effetto maggiore sulla concentrazione degli inquinanti?